In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import torch
import os
from numba import cuda 

from helpers.composite_helpers import *

In [2]:
 os.environ["CUDA_VISIBLE_DEVICES"]="1"

device = cuda.get_current_device()
device.reset()

# set the number of threads that pytorch will use
torch.set_num_threads(2)

# set gpu device
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu")
print( "Using device: " + str( device ), flush=True)


oversample = 1
num_signal_to_inject = 500

seed = 1


curtains_dir = "/global/home/users/rrmastandrea/FETA/"

n_features = 5

dataset_config_string = f"LHCO_{num_signal_to_inject}sig_f/"
STS_config_sting = f"LHCO_STS/"

exp_dir = os.path.join(curtains_dir, dataset_config_string)
data_dir = os.path.join(exp_dir, "data")

Using device: cuda


In [3]:
classif_train_sim = ToyDataset(data_dir, "classif_train_sim.npy")
classif_train_dat = ToyDataset(data_dir, "classif_train_dat.npy")


# load in the reverse rescales
path_to_minmax = "/global/home/users/rrmastandrea/FETA/LHCO_STS/data/col_minmax.npy"
col_minmax = np.load(path_to_minmax)


# dataset generation parameters
context_endpoints = (2500, 4500)


bands_dict = {"ob1": [2500, 2900],
              "sb1": [2900, 3300],
              "sr" : [3300, 3700],
              "sb2": [3700, 4100],
              "ob2": [4100, 4500]}

binning_scheme = np.linspace(-3.5, 3.5, 50)


In [4]:
num_layers_BD_sim = 1
num_hidden_features_BD_sim = 128
num_blocks = 15

hyperparameters_dict_BD_sim = {"n_epochs":100,
                              "batch_size": 128,
                              "lr": 0.0001,
                              "weight_decay": 0.0001}


loc_id_BD_sim = f"BD_sim_Masked_PRQ_AR_{num_layers_BD_sim}layers_{num_hidden_features_BD_sim}hidden_{num_blocks}blocks_{seed}seed"
BD_sim_training_dir = os.path.join(exp_dir, f"saved_models_{loc_id_BD_sim}/")
BD_sim_samples_dir = os.path.join(BD_sim_training_dir, f"npy_samples/")

config_string_BD_sim = "epochs{0}_lr{1}_wd{2}_bs{3}".format(hyperparameters_dict_BD_sim["n_epochs"], hyperparameters_dict_BD_sim["lr"], hyperparameters_dict_BD_sim["weight_decay"], hyperparameters_dict_BD_sim["batch_size"])
checkpoint_path_BD_sim = os.path.join(BD_sim_training_dir, f"BDSIM_{config_string_BD_sim}")

num_layers_s2d = 2
num_nodes_s2d = 16

hyperparameters_dict_dat_props = {"n_epochs": 100,
                              "batch_size": 256,
                              "lr": 0.0005,
                              "weight_decay": 0.00001}
    


loc_id_s2d = f"PRQ_Coupling_{num_layers_s2d}layers_{num_nodes_s2d}nodes_{seed}seed"
# training dir is inside the BD dir
dat_props_dir = os.path.join(BD_sim_training_dir, f"saved_models_{loc_id_s2d}/")
oversampling_dir = os.path.join(dat_props_dir, f"oversampling_{oversample}/") 

os.makedirs(oversampling_dir, exist_ok = True)


In [5]:

# Load in BD and s2d models
config_string_BD_sim = "epochs{0}_lr{1}_wd{2}_bs{3}".format(hyperparameters_dict_BD_sim["n_epochs"], hyperparameters_dict_BD_sim["lr"], hyperparameters_dict_BD_sim["weight_decay"], hyperparameters_dict_BD_sim["batch_size"])
config_string_dat_props = "epochs{0}_lr{1}_wd{2}_bs{3}".format(hyperparameters_dict_dat_props["n_epochs"], hyperparameters_dict_dat_props["lr"], hyperparameters_dict_dat_props["weight_decay"], hyperparameters_dict_dat_props["batch_size"])

checkpoint_path_BD_sim = os.path.join(BD_sim_training_dir, f"BDSIM_{config_string_BD_sim}")
checkpoint_path_dat_props = os.path.join(dat_props_dir, f"TRANS_{config_string_dat_props}")


print("Loading the models ...")
print()
flow_BD_sim = torch.load(f"{checkpoint_path_BD_sim}_best_model.pt")
flow_dat_props = torch.load(f"{checkpoint_path_dat_props}_best_model.pt")

# send networks to device
flow_BD_sim.to(device)
flow_dat_props.to(device)

# freeze the trained models
for param in flow_BD_sim.parameters():
    param.requires_grad = False
flow_BD_sim.eval()

for param in flow_dat_props.parameters():
    param.requires_grad = False
flow_dat_props.eval()


bands_to_sample = ["sr"]


# Apply the flow to BD
SB_sim_samples, SB_BD_samples = make_BD_samples_dict(bands_to_sample, bands_dict, n_features, classif_train_sim, col_minmax, flow_BD_sim, device, oversample)
SB_trans_BD_samples = transform_sim_to_dat_2step(flow_dat_props, SB_BD_samples, device)
SB_trans_BD_samples = np.hstack((SB_trans_BD_samples, np.reshape(SB_BD_samples[:,-1], (-1, 1))))

bands_edges = [bands_dict[band] for band in bands_to_sample]
SB_dat_samples = classif_train_dat.pull_from_mass_range(bands_edges)
SB_dat_samples = minmaxscale(SB_dat_samples.data, col_minmax, lower = -3, upper = 3, forward = True)


print(SB_sim_samples.shape, SB_trans_BD_samples.shape, SB_dat_samples.shape)

# BD
np.save(os.path.join(oversampling_dir, f"BD.npy"), SB_BD_samples)

# transBD
np.save(os.path.join(oversampling_dir, f"transBD.npy"), SB_trans_BD_samples)

# dat_sideband
np.save(os.path.join(oversampling_dir, f"DAT_SB.npy"), SB_dat_samples)



Loading the models ...

Oversampling: 1
(121978, 6) (121978, 6) (121732, 6)
